In [2]:
import numpy as np
import os
from keras.layers import Conv3D, MaxPooling3D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Activation, ZeroPadding3D, TimeDistributed, LSTM, GRU, Reshape
#from keras.utils import plot_model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import gc
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
from sklearn.preprocessing import LabelEncoder
import sklearn
import seaborn as sns
import tensorflow as tf
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from prettytable import PrettyTable
from matplotlib import pyplot as plt
import time

# Model Building

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, TimeDistributed, LSTM, Dense, BatchNormalization, Flatten

# Define constants
IMAGE_SIZE = (96, 72)
BATCH_SIZE = 16
EPOCHS = 100
NUM_CLASSES = 17
TRAIN_DATA_DIR = "D:\\New Dataset Lip Movement Projec\\FINAL YEAR PROJECT\\output_denoised_image_folder"

# Define data generator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Load and preprocess the data
train_generator = datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


# Define the CNN-RNN model
model_cnn_rnn = Sequential()

# CNN block
model_cnn_rnn.add(Conv2D(64, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)))
model_cnn_rnn.add(MaxPooling2D((2, 2)))
model_cnn_rnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn_rnn.add(MaxPooling2D((2, 2)))
model_cnn_rnn.add(Conv2D(256, (3, 3), activation='relu'))
model_cnn_rnn.add(MaxPooling2D((2, 2)))
model_cnn_rnn.add(Conv2D(128, (3, 3), activation='relu'))
model_cnn_rnn.add(MaxPooling2D((2, 2)))

# Time-distributed layer to process each frame independently
model_cnn_rnn.add(TimeDistributed(Flatten()))

# RNN block
model_cnn_rnn.add(LSTM(64, return_sequences=True))
model_cnn_rnn.add(LSTM(32))

# Fully connected layer
model_cnn_rnn.add(Dense(512, activation='relu'))
model_cnn_rnn.add(Dense(NUM_CLASSES, activation='softmax'))

# Compile the model
model_cnn_rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_cnn_rnn.summary()


Found 246 images belonging to 17 classes.
Found 52 images belonging to 17 classes.





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 94, 70, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 47, 35, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 45, 33, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 22, 16, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 14, 256)       147712    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 7, 256)      

# Model Training

In [5]:
# Start the timer
start_time = time.time()

# Train the model
history_cnn_rnn = model_cnn_rnn.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

model_cnn_rnn.save('denoised_cnn_rnn.h5')
# Stop the timer
end_time = time.time()

# Compute the computational time
comp_time = end_time - start_time

# Evaluate the model
train_loss_cnn_rnn, train_acc_cnn_rnn = model_cnn_rnn.evaluate(train_generator)
val_loss_cnn_rnn, val_acc_cnn_rnn = model_cnn_rnn.evaluate(validation_generator)

print(f'Training Accuracy (CNN-RNN): {train_acc_cnn_rnn * 100}')
print(f'Validation Accuracy (CNN-RNN): {val_acc_cnn_rnn * 100}')
print(f'Training time (CNN-RNN): {comp_time}')

Epoch 1/100
15/15 [==============================] - 6s 149ms/step - loss: 2.8356 - accuracy: 0.0348 - val_loss: 2.8305 - val_accuracy: 0.0833
Epoch 2/100
15/15 [==============================] - 1s 89ms/step - loss: 2.8321 - accuracy: 0.0696 - val_loss: 2.8283 - val_accuracy: 0.0833
Epoch 3/100
15/15 [==============================] - 1s 86ms/step - loss: 2.8318 - accuracy: 0.0478 - val_loss: 2.8252 - val_accuracy: 0.0833
Epoch 4/100
15/15 [==============================] - 1s 88ms/step - loss: 2.8305 - accuracy: 0.0565 - val_loss: 2.8224 - val_accuracy: 0.1250
Epoch 5/100
15/15 [==============================] - 1s 89ms/step - loss: 2.8272 - accuracy: 0.0696 - val_loss: 2.8177 - val_accuracy: 0.0833
Epoch 6/100
15/15 [==============================] - 1s 92ms/step - loss: 2.8274 - accuracy: 0.0652 - val_loss: 2.8195 - val_accuracy: 0.1458
Epoch 7/100
15/15 [==============================] - 1s 88ms/step - loss: 2.8081 - accuracy: 0.0696 - val_loss: 2.8138 - val_accuracy: 0.1042
Epoch

c:\Users\ksaqu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 0s 20ms/step - loss: 0.0017 - accuracy: 1.0000
Training Accuracy (CNN-RNN): 100.0
Validation Accuracy (CNN-RNN): 100.0
Training time (CNN-RNN): 143.70097374916077


# Plot Accuracy